In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import operator
import gc

In [2]:
PATH = '../data/'

PROP_DATA_PATH = PATH + 'Property dataset/'
TRN_DATA_PATH = PATH + 'training dataset/'
TEST_DATA_PATH = PATH + 'Test dataset/'

##### Accounts Profiles

This table has information on customers/accounts.   
These are the accounts of whom we are marketing the properties for sale.

###### Profiles are created for every account(or customer) on the basis of following continuous features:
   * active_deals
   * active_deals
   * num_deals_as_client
   * num_deals_as_investor
   * number_of_properties
   * number_of_related_deals
   * number_of_related_properties
   * number_of_won_deals_as_client

###### and, following categorical features:
   * buyer_book
   * servicing_contract
   * cmbs
   * consultant
   * correspondent
   * foreign
   * master_servicer
   * lender_book
   * loan_sales_book
   * loan_servicing

In [3]:
accounts_table = pd.read_csv(TRN_DATA_PATH+'Accounts.csv')

accounts_table.head()

accounts_table.describe()

accts_table_cnts_features = [
                                'active_deals',
                                'activity_count',
                                'num_deals_as_client',
                                'num_deals_as_investor',
                                'number_of_properties',
                                'number_of_related_deals',
                                'number_of_related_properties',
                                'number_of_won_deals_as_client'
                            ]

accts_table_bool_vars = [
                            'buyer_book',
                            'servicing_contract',
                            'cmbs',
                            'consultant',
                            'correspondent',
                            'foreign',
                            'master_servicer',
                            'lender_book',
                            'loan_sales_book',
                            'loan_servicing'
                        ]

# for var in accts_table_bool_vars:
#     print(accounts_table[var].value_counts())

for var in accts_table_bool_vars:
    accounts_table[var].replace([True,False],[1,0],inplace=True)

accounts_table.head()

accts_ids = accounts_table.id_accs.values.tolist()

accts_profiles = accounts_table[accts_table_cnts_features+accts_table_bool_vars].values.astype('float')

True     2189
False     567
Name: buyer_book, dtype: int64
False    2700
True       56
Name: servicing_contract, dtype: int64
False    2749
True        7
Name: cmbs, dtype: int64
False    2756
Name: consultant, dtype: int64
False    2741
True       15
Name: correspondent, dtype: int64
False    2578
True      178
Name: foreign, dtype: int64
False    2754
True        2
Name: master_servicer, dtype: int64
False    1509
True     1247
Name: lender_book, dtype: int64
True     1792
False     964
Name: loan_sales_book, dtype: int64
False    2755
True        1
Name: loan_servicing, dtype: int64


##### Properties profiles

This database contains the universal list of properties and its details

In [4]:
property_data = pd.read_csv(PROP_DATA_PATH+'Properties.csv')
property_data.drop('id_deals',axis=1,inplace=True)
display(property_data.head())
property_data_cnt_feats = [
                            'price_per_sq_ft',
                            'num_buildings',
                            'num_floors',
                            'num_parking_spaces',
                            'sale_amount__c',
                            'size_sf',
                            'year_built'
                        ]

# features_to_impute = [
#                         'price_per_sq_ft',
#                         'num_buildings',
#                         'num_floors',
#                         'num_parking_spaces',
#                         'sale_amount__c',
#                         'size_sf',
#                         'year_built'
#                      ]

# for feat in property_data_cnt_feats:
#     temp = property_data.groupby(['city'])[feat].agg('mean').reset_index()
#     cities = temp['city'].values.tolist()
#     feat_means = temp[feat].values.tolist()
#     for ct in cities:
#         property_data.loc[property_data['city']==ct,feat] = temp.loc[temp['city']==ct][feat].values[0]

property_ids = property_data.id_props.values.tolist()

property_profiles = property_data[property_data_cnt_feats].fillna(0).values

def get_property_feature_vector(idx):
    property_feat_vec = property_profiles[idx]
    return property_feat_vec

def get_similar_accounts(acct_id):
    acct_id_idx = accts_ids.index(acct_id)
    acct_id_feature_vector = accts_profiles[acct_id_idx].astype('float')
    reccs = [(i,scipy.spatial.distance.cosine(acct_id_feature_vector,apf.astype('float')))\
             for i,apf in enumerate(accts_profiles)]
    reccs.sort(key=operator.itemgetter(1))
    return reccs

# array([2.22280e+02, 1.00000e+00, 2.00000e+00, 4.00000e+02, 2.25000e+07,
#        1.01223e+05, 2.00100e+03])

# array([4.29969e+03, 0.00000e+00, 0.00000e+00, 0.00000e+00, 8.00000e+07,
#        1.86060e+04, 2.00300e+03])

,id_props,building_status,building_tax_expenses,city,class,county,sale_status,portfolio,market,num_buildings,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,City-0488,Class B,County-0172,N,False,Market-0059,1.0,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,7/10/2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,City-1975,Class A,County-0465,Y,True,Market-0239,NaN,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",City-1415,Class B,County-0452,N,False,Market-0165,1.0,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,City-1415,Class A,County-0452,N,False,Market-0165,1.0,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,City-0941,NaN,County-0284,N,False,Market-0106,NaN,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


In [5]:
# opportunities_table = pd.read_csv(TRN_DATA_PATH+'Opportunities.csv')

# deal_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Deals_to_Properties.csv')

##### Accounts profiling on the basis of Property Features

In [6]:
accounts_to_properties_table = pd.read_csv(TRN_DATA_PATH+'Accounts_properties.csv')
accounts_to_properties_table.drop('id_deals',axis=1,inplace=True)

d1 = pd.merge(accounts_to_properties_table,property_data,on=['id_props'],how='left')

# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
        {'groupby': ['id_accs'], 'select': 'price_per_sq_ft', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_buildings', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_floors', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'num_parking_spaces', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'sale_amount__c', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'size_sf', 'agg': 'mean'},
        {'groupby': ['id_accs'], 'select': 'year_built', 'agg': 'mean'},
    ]

# Apply all the groupby transformations
for spec in GROUPBY_AGGREGATIONS:
    
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    # print(new_feature)
    # Info
    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
    ))
    
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    # print(all_features)
    # Perform the groupby
    gp = d1[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
    # print(gp)
    d1 = d1.merge(gp, on=spec['groupby'], how='left')
     # Clear memory
    del gp
    gc.collect()

d11 = d1[
            [
                'id_accs',
                'id_accs_mean_price_per_sq_ft',
                'id_accs_mean_num_buildings',
                'id_accs_mean_num_floors',
                'id_accs_mean_num_parking_spaces',
                'id_accs_mean_sale_amount__c',
                'id_accs_mean_size_sf',
                'id_accs_mean_year_built'
            ]
        ]

d11_dash = d11.drop_duplicates()

acc_profiles_wrt_property_features = d11_dash.fillna(0).values[:,1:]

acc_profiles_wrt_property_features.shape

Grouping by ['id_accs'], and aggregating price_per_sq_ft with mean
Grouping by ['id_accs'], and aggregating num_buildings with mean
Grouping by ['id_accs'], and aggregating num_floors with mean
Grouping by ['id_accs'], and aggregating num_parking_spaces with mean
Grouping by ['id_accs'], and aggregating sale_amount__c with mean
Grouping by ['id_accs'], and aggregating size_sf with mean
Grouping by ['id_accs'], and aggregating year_built with mean


(2727, 7)

In [7]:
test_data = pd.read_csv(TEST_DATA_PATH+'Test_Data.csv')

test_data.head()

ids = list(test_data['id_accs'].values)

exclude = set(list(accounts_to_properties_table.id_props.unique()))

print(len(exclude))

63001


In [16]:
def filter_similar_accts(tuples_list):
    uniq = {}
    x = np.array([each[1] for each in tuples_list])
    p = np.percentile(x,1)
    print("accounts_percentile",p)
    for tl in tuples_list:
        if tl[1] not in uniq:
            uniq[tl[1]] = tl[0]     
        if tl[1] > p:
            break
    return uniq

# def filter_similar_accts_updated(tuples_list):
#     sa = []
#     x = np.array([each[1] for each in tuples_list])
#     p = np.percentile(x,0.5)
#     print("accounts_percentile",p)
#     sa = [tl[0] for tl in tuples_list if tl[1] <= p]
#     return sa

results = {}

for i,_id in enumerate(ids):
    most_similar_accts = get_similar_accounts(_id)
    print(len(most_similar_accts))
    filtered_most_similar_accts = filter_similar_accts(most_similar_accts)
    filtered_most_similar_accts = filtered_most_similar_accts.values()
#     filtered_most_similar_accts = filter_similar_accts_updated(most_similar_accts)
    print(len(filtered_most_similar_accts))
    print(filtered_most_similar_accts)
    top_reccs = []
    for most_similar_acct in filtered_most_similar_accts:
        if most_similar_acct > 2727:
            continue
        v1 = None
        v1 = acc_profiles_wrt_property_features[most_similar_acct].astype('float')
        reccs = []
        reccs = [(i,scipy.spatial.distance.cosine(v1,pf))\
                 if not np.isnan(scipy.spatial.distance.cosine(v1,pf)) else (i,0)
                 for i,pf in enumerate(property_profiles)]
        reccs.sort(key=operator.itemgetter(1))
        reccs = [r for r in reccs if r[1]>0]
        x = np.array([each[1] for each in reccs])
        if x.size==0:
            continue
        p = np.percentile(x,1.1)
        top_reccs.extend([property_ids[r[0]] for r in reccs if r[1]<=p])
    results[_id] = list(set(top_reccs)-exclude)
    print(len(results[_id]))

2756
accounts_percentile 1.330248456610228e-07
29
dict_values([79, 367, 1061, 426, 221, 1911, 2125, 1297, 1705, 1102, 1438, 1278, 29, 176, 1615, 315, 1703, 451, 586, 1032, 1371, 1468, 547, 755, 127, 140, 2153, 1612, 797])


/home/chandrashekhar/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


622
2756
accounts_percentile 1.0244015202776069e-06
29
dict_values([6, 854, 1961, 22, 98, 470, 1395, 2729, 2136, 354, 2339, 227, 83, 2349, 24, 2034, 162, 122, 2132, 1203, 2139, 2555, 1680, 61, 2052, 479, 85, 287, 1164])
801
2756
accounts_percentile 9.680130415135047e-07
29
dict_values([22, 24, 85, 54, 6, 229, 479, 20, 227, 98, 2729, 924, 162, 1164, 854, 2164, 1961, 46, 2692, 1680, 1171, 333, 2311, 2555, 133, 683, 2132, 145, 1160])
613
2756
accounts_percentile 7.712776423252076e-07
29
dict_values([200, 393, 382, 1842, 775, 237, 244, 2237, 15, 175, 290, 443, 203, 1800, 1156, 5, 1641, 2133, 1924, 1576, 90, 359, 649, 1784, 2216, 432, 654, 774, 1295])
684
2756
accounts_percentile 0.0001269809498002761
29
dict_values([266, 820, 741, 727, 1508, 1309, 2350, 248, 1209, 846, 213, 730, 1109, 860, 437, 1956, 1701, 16, 590, 1099, 2113, 1916, 182, 2232, 2547, 2295, 348, 2535, 261])
724
2756
accounts_percentile 8.765306495961412e-07
29
dict_values([227, 98, 574, 438, 2729, 2052, 2139, 1572, 122, 1683

In [17]:
import csv
with open('subm18.csv','w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['id_accs','id_prop'])
    for resid,res in results.items():
        for r in res:
            row = [resid]+[r]
            writer.writerow(row)